In [1]:
from os import supports_follow_symlinks
import nltk
import csv
import pathlib
import json
import numpy as np

nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/dgmneto/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
file_list = ['train', 'test','dev']
subject = ['Movies_1', 'Movies_2', 'Movies_3']

In [3]:
def get_entities(frames):
    entities = {}
    for frame in frames:
        for action in frame['actions']:
            if action['slot'] == 'intent':
                continue
            values = action['values']
            for value in values:
                tokens = nltk.word_tokenize(value)
                entity = action['slot']
                for idx, word in enumerate(tokens):
                    if idx == 0:
                        wordtype = ('B-' + entity)
                    else:
                        wordtype = ('I-' + entity)
                    entities[word] = wordtype
    return entities

In [6]:
list_entities=[]
def get_entity_list(json_file):
    phrase_to_entities = []
    for item in json_file:
        if len(item['services']) > 1 or not np.any(np.in1d(subject, item['services'])):
            continue
        for turn in item['turns']:
            if turn['speaker'] == 'SYSTEM':
                continue
            splitted_text = nltk.word_tokenize(turn['utterance'])
            entities = get_entities(turn['frames'])
            for word in splitted_text:
                if word in entities:
                    phrase_to_entities.append([word, entities[word]])
                else:
                    phrase_to_entities.append([word, 'o'])
                
            phrase_to_entities.append(['', ''])
    return phrase_to_entities
    

In [9]:
for file_name in file_list:
    f = open(f'./extraction/{file_name}_dataset.csv', 'w')
    writer = csv.writer(f)
    writer.writerow(['Word', 'Entity'])
    for path in pathlib.Path(f'../dstc8-schema-guided-dialogue/{file_name}').iterdir():
        if path.is_file() and not path.match('*schema.json'):
            file = open(path, 'r')
            json_file = json.loads(file.read())
            word_ent_list = get_entity_list(json_file)
            for word_ent in word_ent_list:
                writer.writerow(word_ent)
    file.close()
    f.close()


In [11]:
import pandas as pd

In [13]:
train_ds = pd.read_csv('./extraction/train_dataset.csv')
test_ds  = pd.read_csv('./extraction/test_dataset.csv')
valid_ds   = pd.read_csv('./extraction/dev_dataset.csv')

In [21]:
t1= train_ds.groupby('Entity').count()
t2= test_ds.groupby('Entity').count()
t3= valid_ds.groupby('Entity').count()

In [22]:
t1

,Word
Entity,
B-genre,100
B-location,308
B-movie_name,292
B-show_date,309
B-show_type,153
B-theater_name,163
I-genre,1
I-location,168
I-movie_name,363


In [23]:
t3

,Word
Entity,
B-cast,13
B-directed_by,25
B-genre,33
B-location,97
B-movie_name,90
B-number_of_tickets,43
B-show_date,105
B-show_time,95
B-show_type,80
